# Laboratorio 2 - Reglas de Asociación y Patrones Secuenciales

## Grupo 1 - Paula Hípola Gómez, José Ignacio Navas Sanz y Belén Ortega Pérez

## Entregable 2 - Patrones Secuenciales

### Importamos las librerias con las que trabajaremos.

In [1]:
%config IPCompleter.greedy=True

import numpy as np                        #Esta libreria nos permite trabajar con numericos
import pandas as pd                       #Pandas nos permite trabajar con datasets y csv
import matplotlib.pyplot as plt           #Las dos librerias anteriores nos permiten pintar las funciones
from sklearn import preprocessing         #Nos sirve para normalizar los datos

from gsppy.gsp import GSP                     #Librería para poder implementar Generalized Sequential Patterns
from mlxtend.frequent_patterns import apriori #Libería relacionada con A priori

### 1. El conjunto de datos spotify_top10.csv contiene el ranking diario de las 10 canciones más reproducidas en Spotify cada día por país. Analizando esta información se pueden extraer conclusiones sobre cómo evoluciona la popularidad de las canciones. Este estudio se puede llevar a cabo aplicando el algoritmo Generalized Sequential Patterns utilizando la implementación de éste disponible en la librería gsppy. 
#### Prueba al menos dos configuraciones de soporte diferentes. Para una de ellas, interpreta algunos de los patrones secuenciales que te resulten curiosos. 

Añadimos el dataset con el que vamos a trabajar. En este caso vamos a utilizar una colección de datos que contiene el top 10 canciones escuchadas cada día por país. 

In [2]:
data = pd.read_csv('spotify_top10.csv')
data.head()                               #Función para ver las primeras instancias del dataset

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,ec
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,ec
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,ec
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,ec
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,ec


Lo primero que debemos llevar a cabo es una limpieza del dataset. Para ello, primero averiguaremos con cuántos missing values y valores duplicados cuenta. 

**Datos duplicados**

In [3]:
duplicates_data = data.duplicated().sum()   #Cuenta el número de datos duplicados
print('Hay',duplicates_data,'datos duplicados')
data.drop_duplicates(inplace = True)        #Eliminamos los datos duplicados

Hay 0 datos duplicados


**Missing values**

In [4]:
missing_data1 = data.isnull().sum().sum()  #Cuenta el total de missing values en el dataset
missing_data2 = data.isnull().sum()        #Cuenta el total de missing values por atributo
print('Hay',missing_data1,' valores nulos')
print('\nDistribuido en columnas:')
print(missing_data2)

Hay 354  valores nulos

Distribuido en columnas:
Position        0
Track Name    177
Artist        177
Streams         0
URL             0
Date            0
Region          0
dtype: int64


Contamos con registros en los que el _Track Name_ y el _Artist_ son nulos. A pesar de tener estos valores vacíos, siempre tenemos una URL que nos lleva a una canción de Spotify. 

Hemos decidido copiar la URL de los registros con campos vacíos porque si aparecen en este conjunto de datos es porque en algún momento estubieron en la plataforma. Puede ser que hayan decidido dejar de compartir su contenido a Spotify y es por esto que no aparecen estos datos. 

In [5]:
#En los campos donde el Track Name es nulo, copiamos la URL.
data.loc[(data["Track Name"].isnull())&(data["URL"].notnull()),"Track Name"] = data["URL"]
data = data.replace(np.nan, '', regex=True)
#Volvemos a mostrar los datos nulos por atributo para demostrar que no quedan valores nulos en el nombre de la canción
missing_data2 = data.isnull().sum()
print(missing_data2)

Position      0
Track Name    0
Artist        0
Streams       0
URL           0
Date          0
Region        0
dtype: int64


In [6]:
changes = data.loc[data['Track Name'] == data['URL']]
changes.head()                                      #Mostramos ejemplos de como hemos suistituido los missing values

,Position,Track Name,Artist,Streams,URL,Date,Region
3094,5,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,16767,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-09,ec
3104,5,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,19114,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-10,ec
3115,6,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,21390,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-11,ec
3125,6,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,19644,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-12,ec
3136,7,https://open.spotify.com/track/1YqcGlCHNquxBhl...,,16155,https://open.spotify.com/track/1YqcGlCHNquxBhl...,2017-11-13,ec


#### Tenemos que elegir una región para ver como se evolucionan las canciones en esa zona. Si no nos ceñimos a una zona, somos incapaces de dar unos datos certeros y encontrar patrones que estamos intentando de descubrir. Si somos capaces de descubrir patrones de como se comportan los usuarios podríamos realizar un buen algoritmo de recomendaciones.

### Seleccionamos el TOP10 Global

In [7]:
globales = data[data['Region'] == 'global']
globales

,Position,Track Name,Artist,Streams,URL,Date,Region
174675,1,Starboy,The Weeknd,3135625,https://open.spotify.com/track/5aAx2yezTd8zXrk...,2017-01-01,global
174676,2,Closer,The Chainsmokers,3015525,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,2017-01-01,global
174677,3,Let Me Love You,DJ Snake,2545384,https://open.spotify.com/track/4pdPtRcBmOSQDlJ...,2017-01-01,global
174678,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,https://open.spotify.com/track/5knuzwU65gJK7IF...,2017-01-01,global
174679,5,One Dance,Drake,2259887,https://open.spotify.com/track/1xznGGDReH1oQq0...,2017-01-01,global
...,...,...,...,...,...,...,...
178360,6,Wolves,Selena Gomez,2962789,https://open.spotify.com/track/7EmGUiUaOSGDnUU...,2018-01-09,global
178361,7,Him & I (with Halsey),G-Eazy,2806086,https://open.spotify.com/track/5k38wzpLb15Ygnc...,2018-01-09,global
178362,8,Échame La Culpa,Luis Fonsi,2667301,https://open.spotify.com/track/1zsG4eaZmkA1dvj...,2018-01-09,global
178363,9,Let You Down,NF,2610265,https://open.spotify.com/track/6mrKP2jyIQmM0rw...,2018-01-09,global


### Agrupamos las transacciones por fecha en la región elegida

In [8]:
transactions = globales.groupby('Date')['Track Name'].agg(lambda x: list(x))
pd.DataFrame(transactions)

,Track Name
Date,
2017-01-01,"[Starboy, Closer, Let Me Love You, Rockabye (f..."
2017-01-02,"[Starboy, Closer, Rockabye (feat. Sean Paul & ..."
2017-01-03,"[Starboy, Closer, Rockabye (feat. Sean Paul & ..."
2017-01-04,"[Starboy, Closer, Rockabye (feat. Sean Paul & ..."
2017-01-05,"[Starboy, Closer, Rockabye (feat. Sean Paul & ..."
...,...
2018-01-05,"[rockstar, Havana, River (feat. Ed Sheeran), N..."
2018-01-06,"[rockstar, Havana, River (feat. Ed Sheeran), N..."
2018-01-07,"[rockstar, Havana, River (feat. Ed Sheeran), N..."


##  GSP - Generalized Sequential Patterns

In [9]:
evaluation = GSP(transactions).search(0.3)
print(evaluation)

[{('Shape of You',): 232, ('Despacito - Remix',): 135, ('Unforgettable',): 114, ('New Rules',): 147, ('Havana',): 129, ('rockstar',): 116}]


#### En la región global, las canciones que más fluctúan por la lista de Top 10 son, Shape of You que aparece 232 veces, seguida de Unforgettable con 147. El Top-3 lo completa Despacito - Remix con 135 veces.

#### El problema de poner un soporte de 0.3 es que estamos perdiendo los datos de las transacciones que se producen juntas, es decir, sólo somos capaces de ver cómo se comportan los usuarios al escuchar las canciones de manera individual pero somos incapaces de realizar un seguimiento o una traza de cómo aparecen las canciones en nuestro set de datos, si escuchar A implica luego escuchar B y posteriormente C.

In [10]:
evaluation2 = GSP(transactions).search(0.1)
print(evaluation2)

[{('Starboy',): 44, ('Rockabye (feat. Sean Paul & Anne-Marie)',): 58, ('Closer',): 38, ('I Don’t Wanna Live Forever (Fifty Shades Darker) - From "Fifty Shades Darker (Original Motion Picture Soundtrack)"',): 76, ("Say You Won't Let Go",): 45, ('Shape of You',): 232, ('Castle on the Hill',): 68, ('Paris',): 75, ('Despacito (Featuring Daddy Yankee)',): 102, ("It Ain't Me (with Selena Gomez)",): 98, ('Something Just Like This',): 84, ("That's What I Like",): 59, ('Perfect',): 98, ('Stay (with Alessia Cara)',): 75, ('Passionfruit',): 40, ('Mask Off',): 59, ('HUMBLE.',): 80, ('Despacito - Remix',): 135, ("I'm the One",): 108, ('Swalla (feat. Nicki Minaj & Ty Dolla $ign)',): 40, ('Attention',): 91, ('Unforgettable',): 114, ('Strip That Down',): 59, ('Wild Thoughts',): 79, ('2U (feat. Justin Bieber)',): 75, ('Feels',): 74, ('Mi Gente',): 102, ('New Rules',): 147, ('Look What You Made Me Do',): 44, ('Havana',): 129, ('1-800-273-8255',): 56, ('Too Good At Goodbyes - Edit',): 54, ('Too Good At G

#### Si bajamos el soporte a 0.1 obtenemos más resultados. En este caso podemos observar como tenemos una serie de canciones que salen solas, mientras que tenemos otro conjunto de canciones que van juntas.

#### En lo que a las canciones sueltas respecta, tenemos que Despacito - Ft Daddy Yankee se ha movido por el TOP 10 por sí sola 102 veces mientras que la versión de Despacito - Remix se ha movido 135 veces. También podemos mencionar que Mi Gente y New Rules se han movido 102 y 147 veces respectivamente.

#### Si pasamos a analizar las transacciones que van juntas, tenemos que la transacción que más se ha dado por el ránking es la de 'Rockstar - Havana' con 72 movimientos seguida de 'Shape Of You - HUMBLE' y 'Despacito - Remix - I'm the One' con 51 veces las dos.

#### Podemos sacar un patrón de que los usuarios que escuchan Rockstar escuchan Havana del mismo modo que los que escuchan Shape Of You hacen los mismo con Humble. Es curioso que escuchen a un artista como Ed Sheeran y luego pasen a una canción de un estilo bastante contrario como Humble.

## Conclusión:

### Si hubiésemos bajado el soporte a la hora de realizar el GSP y tuviéramos más datos a lo largo del tiempo, puesto que sólo tenemos los datos del año 2017 y unos pocos de los primeros días de Enero de 2018, podríamos haber obtenido muchas mas secuencias de k-ítemsets relacionados entre sí y poder descubrir mejor los patrones de comportamiento.

### Estos patrones probablemente sirvan para que Spotify te recomiende ciertas canciones que escuchar o incluso te cree listas personalizadas como 'Made For You'. Esto le permite a Spotify trazar tus gustos y de un modo u otro le permitirá mantener alta su tasa de usuarios suscritos.